In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('lr_example').getOrCreate()

In [3]:
from pyspark.ml.regression import LinearRegression

In [5]:
data = spark.read.csv('Ecommerce_Customers.csv', inferSchema=True, 
                     header=True)

In [6]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [9]:
for item in data.head(2)[1]:
    print(item)

hduke@hotmail.com
4547 Archer CommonDiazchester, CA 06566-8576
DarkGreen
31.92627202636016
11.109460728682564
37.268958868297744
2.66403418213262
392.2049334443264


In [10]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [11]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [12]:
assembler = VectorAssembler(inputCols=['Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership'],
                           outputCol='features')

In [13]:
output = assembler.transform(data)

In [14]:
output.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)
 |-- features: vector (nullable = true)



In [15]:
output.select('features').show()

+--------------------+
|            features|
+--------------------+
|[34.4972677251122...|
|[31.9262720263601...|
|[33.0009147556426...|
|[34.3055566297555...|
|[33.3306725236463...|
|[33.8710378793419...|
|[32.0215955013870...|
|[32.7391429383803...|
|[33.9877728956856...|
|[31.9365486184489...|
|[33.9925727749537...|
|[33.8793608248049...|
|[29.5324289670579...|
|[33.1903340437226...|
|[32.3879758531538...|
|[30.7377203726281...|
|[32.1253868972878...|
|[32.3388993230671...|
|[32.1878120459321...|
|[32.6178560628234...|
+--------------------+
only showing top 20 rows



In [16]:
output.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [17]:
final_data = output.select('features', 'Yearly Amount Spent')

In [18]:
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [19]:
train_data, test_data = final_data.randomSplit([0.7, 0.3])

In [20]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                352|
|   mean|  501.7603941708334|
| stddev|   80.0774938827114|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [21]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                148|
|   mean|  493.4956782510945|
| stddev|  77.42696350248406|
|    min|   266.086340948469|
|    max|   666.125591725772|
+-------+-------------------+



In [22]:
lr = LinearRegression(labelCol='Yearly Amount Spent')

In [23]:
lr_model = lr.fit(train_data)

In [24]:
test_results = lr_model.evaluate(test_data)

In [25]:
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
|  10.35737502532379|
|-12.644585124572018|
|-18.318648184310064|
|  -5.05833752709276|
|-21.701904048134224|
|   8.43729920038254|
| -4.436196329746053|
| 2.5094476779983097|
| -5.897826908886373|
| -5.518527717712118|
| -9.622197463067494|
| -2.078284349317414|
|  17.67880060418929|
|  5.793897830549611|
|-2.6253513740827543|
|-18.956947361909045|
|  7.629934280084569|
| -2.791387109555899|
| -5.498202661117318|
|  -9.30750628176878|
+-------------------+
only showing top 20 rows



In [26]:
test_results.rootMeanSquaredError

10.052402190972009

In [27]:
test_results.r2

0.983029304633698

In [28]:
final_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                500|
|   mean|  499.3140382585909|
| stddev|   79.3147815497068|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [29]:
# The rootMeanSquaredError is 10$ and the average is about 500$ and the standard desviation is about $80 so
# the model seems to have a good fitting. Also the R2 indicates that 98% variance
# is explained. This double check is a good indicators to trust in the model.

In [30]:
# Deploy model

In [31]:
unlabeled_data = test_data.select('features')

In [32]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|[29.5324289670579...|
|[30.3931845423455...|
|[30.8162006488763...|
|[30.8364326747734...|
|[31.1239743499119...|
|[31.1695067987115...|
|[31.2681042107507...|
|[31.3091926408918...|
|[31.5147378578019...|
|[31.5257524169682...|
|[31.5261978982398...|
|[31.5761319713222...|
|[31.6005122003032...|
|[31.6548096756927...|
|[31.7216523605090...|
|[31.8164283341993...|
|[31.8209982016720...|
|[31.8530748017465...|
|[31.8745516945853...|
|[31.8854062999117...|
+--------------------+
only showing top 20 rows



In [33]:
predictions = lr_model.transform(unlabeled_data)

In [34]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[29.5324289670579...| 398.2829760473037|
|[30.3931845423455...| 332.5734549277656|
|[30.8162006488763...|284.40498913277906|
|[30.8364326747734...|472.56023795408237|
|[31.1239743499119...|    508.6489578879|
|[31.1695067987115...|418.91923160191027|
|[31.2681042107507...|   427.90672950357|
|[31.3091926408918...| 430.2112701619353|
|[31.5147378578019...| 495.7103149053478|
|[31.5257524169682...|  449.484154527594|
|[31.5261978982398...| 418.7167236554053|
|[31.5761319713222...| 543.3048683386457|
|[31.6005122003032...|461.49405088690764|
|[31.6548096756927...| 469.4695258969989|
|[31.7216523605090...| 350.4022780059554|
|[31.8164283341993...| 520.0794388655654|
|[31.8209982016720...| 417.0453467331288|
|[31.8530748017465...| 462.0765105719079|
|[31.8745516945853...| 397.7834469073848|
|[31.8854062999117...| 399.4107792542443|
+--------------------+------------